In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe History Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.9.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

595e48a82babf64fab7ffcedb22cda4203b8e751


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [8]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [9]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [10]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [11]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "0af26731b01b27e80cfeddf85e8acc2e29c63140241793b3dd289908862e8bd4"


#### The Counterparty

In [12]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [13]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [14]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "40a233dd5adfebf005c8142bbc3af3215b2a94f0203d65b4501cb289ad1658d2"


### Time computations

In [15]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [16]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666208911000


## The Contract

We set the parameters for the ACTUS PAM contract.

In [17]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

In [18]:
yaml2json << EOI > history.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat history.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-19T00:00:00"}


Create the contract and its initial state.

In [19]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party "$PARTY_ADDR" \
  --counter-party "$COUNTERPARTY_ADDR" \
  --actus-terms-file  history.actus \
  --out-contract-file history-1.contract \
  --out-state-file    history-1.state

Since we are testing, we don't really want to wait months or years for timeouts, so we edit the contract file to change the maturity date to 15 minutes from now and the initial deposit to 10 minutes from now.

In [20]:
sed -i \
  -e "s/$(jq '.timeout' history-1.contract)/$((NOW + 10 * MINUTE))/" \
  -e "s/$(($(date -d "$MATURITY_DATE" -u +%s) * SECOND))/$((NOW + 15 * MINUTE))/" \
  history-1.contract

View the contract.

In [21]:
json2yaml history-1.contract

timeout: 1666209511000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  then:
    from_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    pay: 100000000
    then:
      timeout: 1666209811000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

View the initial state.

In [22]:
json2yaml history-1.state

accounts:
- - - address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1


## Create two transactions so that they can be viewed by Marlowe History

### Transaction 1. Create the contract

First we create a `.marlowe` file that contains the initial information needed to run the contract.

In [23]:
marlowe-cli run initialize \
  --contract-file history-1.contract \
  --state-file    history-1.state    \
  --out-file      history-1.marlowe  \
  --print-stats


Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


Submit the transaction.

In [24]:
TX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file history-1.marlowe \
  --required-signer "$PARTY_SKEY" \
  --change-address "$PARTY_ADDR" \
  --out-file history-1.txbody \
  --print-stats \
  --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_1 = $TX_1"


Fee: Lovelace 220457
Size: 1133 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0


The contract is identified by its first UTxO.

In [25]:
CONTRACT_ID="$TX_1#1"
echo "$CONTRACT_ID"

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1


### Transaction 2. Party deposits loan amount

In [26]:
marlowe-cli run prepare \
  --marlowe-file history-1.marlowe \
  --out-file     history-2.marlowe \
  --deposit-account "$PARTY_ADDR" \
  --deposit-party "$PARTY_ADDR" \
  --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
  --invalid-before "$((NOW-5*MINUTE))" \
  --invalid-hereafter "$((NOW+9*MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666208611000},POSIXTime {getPOSIXTime = 1666209451999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666208611000},POSIXTime {getPOSIXTime = 1666209451999}), txInputs = [NormalInput (IDeposit "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" (Token "" "") 100000000)]}

Datum size: 605
Payment 1
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Ada: Lovelace {getLovelace = 100000000}


Submit the transaction.

In [27]:
TX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#1" \
  --marlowe-in-file  history-1.marlowe \
  --marlowe-out-file history-2.marlowe \
  --required-signer "$PARTY_SKEY" \
  --change-address "$PARTY_ADDR" \
  --out-file history-2.txbody \
  --print-stats \
  --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_2 = $TX_2"


Fee: Lovelace 1294779
Size: 14464 / 16384 = 88%
Execution units:
  Memory: 6287250 / 14000000 = 44%
  Steps: 1732252213 / 10000000000 = 17%
TX_2 = c4945e5fb54f173c12b9ded77e41e105ddb14140c26dbf1b893a57589c6a85f2


## Contract history commands

Several `marlowe` commands relate to contract history.

In [28]:
marlowe --help

Usage: marlowe [--history-host HOST_NAME] [--history-command-port PORT_NUMBER] 
               [--history-query-port PORT_NUMBER] 
               [--history-sync-port PORT_NUMBER] [--discovery-host HOST_NAME] 
               [--discovery-query-port PORT_NUMBER] [--tx-host HOST_NAME] 
               [--tx-command-port PORT_NUMBER] (COMMAND | COMMAND | COMMAND)

  Command line interface for managing Marlowe smart contracts on Cardano.

Available options:
  -h,--help                Show this help text
  --history-host HOST_NAME The hostname of the Marlowe Runtime history server.
                           Can be set as the environment variable
                           MARLOWE_RT_HISTORY_HOST (default: "127.0.0.1")
  --history-command-port PORT_NUMBER
                           The port number of the history server's job API. Can
                           be set as the environment variable
                           MARLOWE_RT_HISTORY_COMMAND_PORT (default: 3717)
  --history-query-port 

### Add the contract to Marlowe History

In [29]:
marlowe add "$CONTRACT_ID"

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1


### List the contracts managed by Marlowe History

In [30]:
marlowe ls --show-status

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1 Status: Waiting (SomeMarloweVersion MarloweV1)


### Check removal of the contract from Marlowe History

Remove it.

In [31]:
marlowe rm "$CONTRACT_ID"

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1


See that it has been removed.

In [32]:
marlowe ls

Add it back.

In [33]:
marlowe add "$CONTRACT_ID"

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1


In [34]:
marlowe ls

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1


### List all Marlowe contracts

In [35]:
marlowe ls --all --show-status | wc -l

301


In [36]:
marlowe ls --all --show-status | head

00e6264722d6a02193cea60ad0c055f17fcbf4e69e0dae1f7c96a3eaf352038c#1 Status: not added
02811e36c6cdac4721b53f718c4a1406e09ef0d985f9ad6b7fd676769e2f866c#1 Status: not added
044a07c9b25be511295ac7cda31857a613a39228d0775851ffa69923abd69344#1 Status: not added
045d4ac6f50c57cade58fd697b907ff0418637ba1eadb36392e04d1c5d489b21#1 Status: not added
053a5b3eeeac7629e301cdc73f94d7b5f67af6846f80010b509b6d108191d937#1 Status: not added
0552fc00860d35d45c09b4b43740193579adbeea5097cf33b62201332e3095ee#1 Status: not added
05574d9d1935d2a5d2e3cb15e79d46b076e4c81d85bd2e12940ed47b628001be#1 Status: not added
05990678496287262051fc831ff35903e383ffc564b943de76e78394a8ceb4cd#1 Status: not added
059b32a85f41480ba6b83e210a8999f38d691ef1500286b9d67a769b74597a31#1 Status: not added
061d29872aae0580496aecc8bed3a7a11d743c55d24f6d2c5c03c630cf89e899#1 Status: not added
Marlowe discovery query client failure


<span style="color:red;font-weight:bold">This reports a "Marlowe discovery query client failure" because `head` stops reading input.</span>

### See if there are any failed contracts

In [37]:
marlowe ls --all > all.tmp
marlowe ls --all --show-failed > failed.tmp
diff all.tmp failed.tmp

<span style="color: red; font-weight: bold">I think that <code>--show-failed</code> should <em>only</em> show the failed contracts, not all of them.</span>

### View the basic information about the contract

In [38]:
marlowe log "$CONTRACT_ID"

transaction fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0 (creation)
ContractId:      fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1
SlotNo:          6205723
BlockNo:         286690
BlockId:         43ac040b0d2ca80253ad5bae13832c5ac5ec938b4af514a6abc2891391cfe924
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1



transaction c4945e5fb54f173c12b9ded77e41e105ddb14140c26dbf1b893a57589c6a85f2
ContractId: fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1
SlotNo:     6205732
BlockNo:    286691
BlockId:    ebd93fbca1c95b61c66dda95fea46bb147ba4f1249ae32f2dee3ee251b8cc85a
Inputs:     [NormalInput (IDeposit "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" (Token "" "") 100000000)]




### View the contract itself

In [39]:
marlowe log --show-contract "$CONTRACT_ID"

transaction fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0 (creation)
ContractId:      fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1
SlotNo:          6205723
BlockNo:         286690
BlockId:         43ac040b0d2ca80253ad5bae13832c5ac5ec938b4af514a6abc2891391cfe924
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

## Finish running the contract

### Transaction 3. Counterparty repays the loan's interest

In [40]:
marlowe-cli run prepare \
  --marlowe-file history-2.marlowe \
  --out-file     history-3.marlowe \
  --deposit-account "$COUNTERPARTY_ADDR" \
  --deposit-party "$COUNTERPARTY_ADDR" \
  --deposit-amount "$((INTEREST*FIXED_POINT))" \
  --invalid-before "$((NOW-5*MINUTE))" \
  --invalid-hereafter "$((NOW+9*MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666208611000},POSIXTime {getPOSIXTime = 1666209451999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666208611000},POSIXTime {getPOSIXTime = 1666209451999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" (Token "" "") 2000000)]}

Datum size: 349
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 2000000}


In [41]:
TX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_2#1" \
  --marlowe-in-file  history-2.marlowe \
  --marlowe-out-file history-3.marlowe \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$COUNTERPARTY_ADDR" \
  --out-file history-3.txbody \
  --print-stats \
  --submit 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_3 = $TX_3"


Fee: Lovelace 1233027
Size: 13952 / 16384 = 85%
Execution units:
  Memory: 5815658 / 14000000 = 41%
  Steps: 1565634381 / 10000000000 = 15%
TX_3 = 6f19ed7f9bae49445d64a379d2f537335859a788c1d621b2abedac6e8b78bda3


### Transaction 4. Counterparty repays the loan's principal

In [42]:
marlowe-cli run prepare \
  --marlowe-file history-3.marlowe \
  --out-file     history-4.marlowe \
  --deposit-account "$COUNTERPARTY_ADDR" \
  --deposit-party "$COUNTERPARTY_ADDR" \
  --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
  --invalid-before "$((NOW-5*MINUTE))" \
  --invalid-hereafter "$((NOW+9*MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666208611000},POSIXTime {getPOSIXTime = 1666209451999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666208611000},POSIXTime {getPOSIXTime = 1666209451999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" (Token "" "") 100000000)]}

Datum size: 30
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 100000000}
Payment 2
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 3000000}


In [43]:
TX_4=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_3#1" \
  --marlowe-in-file  history-3.marlowe \
  --marlowe-out-file history-4.marlowe \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$COUNTERPARTY_ADDR" \
  --out-file history-4.txbody \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_4 = $TX_4"


Fee: Lovelace 1078968
Size: 13273 / 16384 = 81%
Execution units:
  Memory: 4210566 / 14000000 = 30%
  Steps: 1127777514 / 10000000000 = 11%
TX_4 = 54054c66bf46a00b16a9443e2da77c2888f8841bd2e549c3b60bdaa627679608


The contract has closed.

In [44]:
jq '.tx.contract' history-4.marlowe | json2yaml

close
...


<span style="color: red; font-weight: bold">The contract has closed, but it looks like Marlowe History thinks it is still open:</span>

In [45]:
marlowe ls --show-status

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1 Status: Waiting (SomeMarloweVersion MarloweV1)


In [46]:
marlowe log "$CONTRACT_ID"

transaction fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0 (creation)
ContractId:      fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1
SlotNo:          6205723
BlockNo:         286690
BlockId:         43ac040b0d2ca80253ad5bae13832c5ac5ec938b4af514a6abc2891391cfe924
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1



transaction c4945e5fb54f173c12b9ded77e41e105ddb14140c26dbf1b893a57589c6a85f2
ContractId: fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1
SlotNo:     6205732
BlockNo:    286691
BlockId:    ebd93fbca1c95b61c66dda95fea46bb147ba4f1249ae32f2dee3ee251b8cc85a
Inputs:     [NormalInput (IDeposit "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" (Token "" "") 100000000)]


transaction 6f19ed7f9bae49445d64a379d2f537335859a788c1d621b2abedac6e8b78bda3
ContractId: fa5906a75134a1eb71b97c534f2b50

In [47]:
marlowe log --show-contract "$CONTRACT_ID"

transaction fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0 (creation)
ContractId:      fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1
SlotNo:          6205723
BlockNo:         286690
BlockId:         43ac040b0d2ca80253ad5bae13832c5ac5ec938b4af514a6abc2891391cfe924
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r" Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Token "" "")
            (NegValue
               (NegValue
                  (Constant 100000000))))
         (Pay Address "addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r"
            (Party Address "addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5")
            (Token "" "")
            (Constant 100000000)
            (When [
               (Case
          

## Cleanup

Stop following the contract.

In [48]:
marlowe rm "$CONTRACT_ID"

fa5906a75134a1eb71b97c534f2b503285dfc107e76ff9fe7086b0694b2c12b0#1


In [49]:
marlowe ls

Consolidate the UTxOs at the addresses.

In [50]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "f94563f8c95e0c6667f9405fbf0d99d04fadb655edcf9faabe2fe52d9ed6fb0e"


In [51]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "4f1871ed4f2ba0333adb1d4b71b7c3664b814c18d50cf1ef3532001576207d65"


Send the funds back to the faucet.

In [52]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "4d1f47687540c7733e3543c76baf0b1ae179f7d6811ff35f8f4acba8efa80e8f"


See that the funds have been returned to the faucet.

In [53]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
